# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3 arxiv==2.3.1 numexpr==2.14.1 wikipedia==1.4.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/a5/33/e9e0f4663e55f72be6408f294a42ef15da3095b8bd014275502e88085e4c/dashscope-1.25.4-py3-none-any.whl (1.3 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/74/74/246f809a3741c21982f985ca0113ec92d3c84896308561cc4414823f6951/langchain_classic-1.0.0-py3-none-any.whl (1.0 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f3/39/ed3121ea3a0c60a0cda6ea5c4c1cece013e8bbc9b18344ff3ae507728f98/langchain-1.1.3-py3-none-any.whl (102 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/f0/a4/c4fde67f193401512337456cabc2148f2c43316e445f5decd9f8806e2992/langchain_community-0.4.1-py3-none-any.whl (2.5 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/d1/11/2b3b4973495fc5f0456ed5c8c88a6ded7ca34c8608c72faafa87088acf5a/langchain_openai-1.1.3-py3-none-any.whl (84 kB)
  Using cached wikipedia-1.4.0-py3-none-any.whl
  Using cached https://pypi.tun

## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

# 2. 旧版 LangChain ReAct Agent 搭建

## 2.1 简介

看完了如何通过基础的 python 代码手搓 ReAct Agent，下面我们来看看如何使用 LangChain 框架来复现该系统。

对于搭建一个基于 LangChain 的 ReAct Agent 系统，我们还是需要以下几个零部件：
- LLM 模型（大脑）
- Memory（记忆）
- 工具（行动能力）
- 提示词模版（思维方式）

只不过这里的零部件我们使用更快速和简便的方式即可实现。LangChain 框架帮助我们将很多的内容写好，我们只需要考虑具体的工具设置以及运行逻辑即可。

## 2.2 LLM 模型

对于模型的搭建我们其实和第三节课的是一样的，只不过是这里的模型我们要用比较强一些的模型 ernie-4.5-turbo-128k ，从而确保其能够有足够强的能力面对复杂场景下的推理。

In [3]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
 model="ernie-4.5-turbo-128k",
 openai_api_key=os.environ.get("OPENAI_API_KEY"),
 base_url="https://aistudio.baidu.com/llm/lmapi/v3" 
)

response = llm.invoke("你好，请介绍一下你自己")

print(response.content)

你好！我是百度公司研发的知识增强大语言模型，我的中文名是文心一言，英文名是ERNIE Bot。我能够与人对话互动，回答问题，协助创作，高效便捷地帮助人们获取信息、知识和灵感。如果你有任何问题，都可以随时告诉我。


## 2.3 Tool 工具

### 2.3.1 内置工具

根据LangChain的官方文档，目前LangChain提供支持的有以下几类工具：
- 搜索工具：用于在线搜索，包括 Bing、Google、DuckDuckGo 等搜索接口。返回内容一般包括：URL、标题、摘要等。
- 代码辅助工具：支持 Python、JavaScript 等语言的代码执行环境，可用于复杂计算或文件处理。
- 生产力工具：用于对接 Gmail、Office365、Slack、Jira 等办公/协作平台，实现任务自动化。
- 网页浏览：用于浏览网页、交互操作、信息抓取（如Hyperbrowser 等）。
- 数据库：支持与数据库交互，包括 SQL、Spark SQL 等数据库的读取与操作。
- 其他常用工具：维基百科、ArXiv、Python REPL

假如我们需要使用 LangChain 内置的工具，我们首先需要使用一个 load_tools 工具，然后在里面写入对应工具的名称：

In [4]:
from langchain_community.agent_toolkits.load_tools import load_tools
tools = load_tools(["arxiv"])

假如我们想添加更多的工具，我们可以在后面不断添加（部分依赖语言模型来执行计算或生成代码需要传入一个 LLM 实例）：

In [5]:
tools = load_tools(["arxiv","llm-math", "wikipedia"], llm=llm)

### 2.3.2 自定义工具

但是在我们当前的 Agent 设定下，我们并不需要使用上 LangChain 里内置的工具。

我们所使用的是我们自己定义的 calculate 工具和 average_dog_weight 工具。

因此下面我们就来看看如何将这些函数转化为 LangChain 中可使用的工具：

In [6]:
def calculate(what):
    return eval(what)

print(calculate("3 + 7 * 2"))   # 返回 17
print(calculate("10 / 4"))      # 返回 2.5

def average_dog_weight(name):
  if name in "Scottish Terrier": 
    return("Scottish Terriers average 20 lbs")
  elif name in "Border Collie":
    return("a Border Collies average weight is 37 lbs")
  elif name in "Toy Poodle":
    return("a toy poodles average weight is 7 lbs")
  else:
    return("An average dog weights 50 lbs")

print(average_dog_weight("Scottish Terrier")) 
# 返回 "Scottish Terriers average 20 lbs"
print(average_dog_weight("Labrador"))     
# 返回 "An average dog weights 50 lbs"

17
2.5
Scottish Terriers average 20 lbs
An average dog weights 50 lbs


在 LangChain里，定义工具的主要方法为使用 @tool 装饰器。这种方式最为简洁，只需在普通函数上方加上 @tool 装饰器,并且在内部加上文档字符串作为工具的介绍以及参数的说明，就能自动将该函数注册为一个可供智能体调用的工具。

系统会自动提取函数的名称、参数说明和文档字符串（上一个提示词里对工具的示例）作为工具的描述（写入提示词中），非常适合快速创建一些简单的功能型工具。例如：计算表达式、查询天气等。

In [7]:
from langchain.tools import tool

@tool
def calculate(what: str) -> str:
    """
    calculate:
    e.g. calculate: 4 * 7 / 3
    Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary
    """
    return str(eval(what))

假如我们不想在函数内部写入文档字符串，我们也可以在 @tool 中添加一个属性 description 去写入同样的内容。

这里的 name 表示工具的名称（假如没有就默认使用函数名称）。description 代表的函数的介绍，也就是与前面文档字符串的作用一致。

In [8]:
from langchain.tools import tool

@tool(description="""
  calculate:
  e.g. calculate: 4 * 7 / 3
  Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary
  """)
def calculate(what: str) -> str:
    return str(eval(what))

在 LangChain v1.0 版本中还新增了一个基于 Pydantic 的输入格式审查工具。

我们可以先定义一个 Pydantic 输入模型，通过 what: str 明确输入的类型是字符串，然后介绍需要输入里面的内容是什么。比如这里介绍的就是输入的是一个字符串格式的数学表达式。

然后再装饰器里我们需要将这个类载入到 @tool 的 args_schema 参数中，然后后续在模型调用工具的时候就能获取这部分信息。

In [9]:
from pydantic import BaseModel, Field

class CalcInput(BaseModel):
    """Input for math calculation"""
    what: str = Field(description="A mathematical expression, e.g., '4 * 7 / 3'")
    
from langchain.tools import tool

@tool(args_schema=CalcInput)
def calculate(what: str) -> str:
    """
    calculate:
    e.g. calculate: 4 * 7 / 3
    Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary
    """
    return str(eval(what))

在定义好两个工具后，我们可以将两个工具组合起来形成工具列表，等待后续传给 ReAct Agent 进行使用：

In [10]:
from langchain.tools import tool
import re

@tool
def calculate(what: str) -> str:
  """
    Use this tool to perform arithmetic calculations.

    Input requirements:
    - The input must be a single-line arithmetic expression
    - Use only numbers and operators: + - * / ( )
    - DO NOT include any text, explanations, markdown, or words like "Observation"

    Examples:
    - 37 + 20
    - 57 * 2
    - (4 * 7) / 3

    This tool returns the numeric result as a string.
  """
  # 1️⃣ 去掉 Markdown
  text = what.replace("```", "").strip()
  # 2️⃣ 只取第一行（🔴 核心）
  first_line = text.splitlines()[0]
  # 3️⃣ 只允许算术字符（🔒 再保险）
  first_line = re.sub(r"[^0-9\.\+\-\*\/\(\) ]", "", first_line)
  return str(eval(first_line))

@tool
def average_dog_weight(name: str) -> str:
  """
    Use this tool to look up the average adult weight of a dog breed.

    Input requirements:
    - The input should ONLY be the dog breed name
    - Do NOT include extra text, questions, or explanations

    Examples:
    - Border Collie
    - Scottish Terrier
    - Toy Poodle

    This tool returns a short sentence stating the average weight in pounds (lbs).
  """
  name = name.lower()
  if "scottish terrier" in name:
    return "Scottish Terriers average 20 lbs"
  elif "border collie" in name:
    return "A Border Collie's average weight is 37 lbs"
  elif "toy poodle" in name:
    return "A Toy Poodle's average weight is 7 lbs"
  else:
    return "An average dog weighs 50 lbs"

tools = [calculate, average_dog_weight]

假如要和默认的工具一起导入的话，就需要：

In [11]:
# tools = load_tools(["arxiv"]) + [calculate, average_dog_weight]

## 2.4 Prompt Template 提示词模版

在 LangChain 中，其实内部提供了很多的提示词模版，我们可以直接调取使用。

这些提示词模版都保存在 LangChain Hub 中，这个平台是由 LangChain 官方推出的一个提示词模板共享与管理平台， 用于集中存放、共享、和复用各类提示词、链以及智能体的配置。 

对于 ReAct 框架，常用的包括：
- hwchase17/react（单轮对话）
- hwchase17/react-chat（多轮对话）

In [12]:
from langchain_classic import hub
prompt = hub.pull("hwchase17/react-chat")
print(prompt)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react-chat', 'lc_hub_commit_hash': '3ecd5f710db438a9cf3773c57d6ac8951eefd2cd9a9b2a0026a65a0893b86a6e'} template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide ra

该提示词主要划分为五部分：
- 系统角色与能力设定
- 工具介绍
- 工具调用方式（ReAct 核心）
- 无需工具时的结束协议
- 运行时上下文注入（形成 ReAct 闭环）

## 2.5 Memory 记忆

对于记忆模块，其实我们在讲 LangChain 框架的时候已经提到过，使用的方法就是 RunnableWithMessageHistory 。

但是这里我们需要注意的是，之前传入到 runnable 参数的是 chain ，也就是一个将 prompt 和 llm 组合在一起的 LCEL 链。

但是在 Agent 中，我们并不能直接使用 chain = prompt | llm 作为可运行的组件（runnable），因为在 prompt 里有很多的参数需要传入。

所以我们需要使用 LangChain 中的 create_react_agent() 将 llm，tools 和 prompt 组装起来，形成一个 agent 大脑。然后再用 AgentExecutor() 成为一个 runnable 组件，让 Agent 真的“执行起来”。



In [13]:
from langchain_classic.agents import AgentExecutor, create_react_agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

可以看到虽然 create_react_agent 和 AgentExecutor 都有 tools 参数，但是前者的主要目的是让模型知道有哪些工具能用（写进 prompt），而后者是让执行器真的能调用这些工具函数。最后 verbose = True 会将内部运行的信息打印出来。

然后我们就可以将这个 runnable 的 Agent 执行器放入到记忆保存的系统当中。

那这里的记忆保存我们就和之前最基础的一样用全量保存的方式实现。但是我们可以根据需要通过 trim_messages 实现记忆的剪裁：

In [14]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

store = {}
def get_session_history(session_id: str):
 if session_id not in store:
  store[session_id] = InMemoryChatMessageHistory()
 return store[session_id]

agent_with_chat_history = RunnableWithMessageHistory(
  agent_executor,
  get_session_history=get_session_history,
  input_messages_key="input",
  history_messages_key="chat_history")

## 2.6 系统组装

当我们准备好了四部分Agent的组件：
- LLM 模型（大脑）
- Memory （记忆）
- 工具 （行动能力）
- 系统提示词（思维方式）

我们就可以将其组装起来，实现真正的Agent流程了。

In [15]:
config = {"configurable": {"session_id": "test-session"}}

print(agent_with_chat_history.invoke({"input": "I have 2 dogs, a border collie and a scottish terrier. What is their combined weight?"}, config)["output"])

print(agent_with_chat_history.invoke({"input": "how about double it?"}, config)["output"])



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: average_dog_weight
Action Input: Border Collie
ObservationA Border Collie's average weight is 37 lbsThought: Do I need to use a tool? Yes
Action: average_dog_weight
Action Input: Scottish Terrier
ObservationScottish Terriers average 20 lbsThought: Do I need to use a tool? Yes
Action: calculate
Action Input: 37 + 20
Observation57Thought: Do I need to use a tool? No
Final Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.

> Finished chain.
The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: calculate
Action Input: 57 * 2
Observation114```
Thought: Do I need to use a tool? No
Final Answer: The combined weight of two Border Collies and two Scottish Terriers is 114 lbs.
```

> Finished chain.
The combined weight of two Border Collies and two Scottish Terriers is 

## 2.7 方法对比

通过对比框架开发和非框架两种方式，我们可以看到：
- LangChain 等框架化的 ReAct 实现，通过高度抽象的组件化设计，将工具调用、提示词构建、推理循环和记忆管理整合到统一的执行流程中。
- 它让开发者能够专注于业务逻辑本身，而无需手动处理底层的 Action 解析、状态传递和多轮对话维护。这种方式更符合现代智能体开发的趋势——模块化、可扩展、可复用、可持久。
- 而非框架的手写 ReAct 实现，则以最直接的形式揭示了智能体推理的“底层机制”。但这种方式对开发者要求更高，难以维护和扩展。

因此，总的来说：
- 框架式实现 → 适用于工程化、教学系统或生产部署，体现了智能体的可扩展与可持续特征。
- 非框架式实现 → 适用于教学演示或研究实验，帮助我们深入理解 ReAct 的核心原理与思维闭环。

